# Exercise 09 - Reasoning with SWRL

In today's exercise we will be using the Python library *owlready2* again to create and reason with rules written in the SWRL language.
A quick overview over the usage of SWRL in *owlready2* can be found in the [documentation](https://owlready2.readthedocs.io/en/latest/rule.html) and a reference for SWRL, including the different build-ins, exists at the [W3 Compendium](https://www.w3.org/submissions/SWRL/).

As an ontology, we will be using the domain of [Pokémon](https://bulbapedia.bulbagarden.net/wiki/Main_Page), similar to last week's exercise.
However, since we needed to make some adaptations to the underlying ontology, please use the provided one found in the Moodle course (PokemonOntology.owl).

## Setup

In [ ]:
# First we install the library
!pip install owlready2

In [ ]:
# Next we import the necessary libraries
from owlready2 import *

In [ ]:
# Continue with loading the ontology and defining the namespace
pkmn_onto = get_ontology("./PokemonOntology.owl").load()

# Check the available classes
print(list(Thing.subclasses()))

## Adding Basic SWRL rules

We now want to add basic SWRL rules to the Pokémon ontology to represent specific information about them. As an example, we want to modell the different relations between the Pokémon types. In the games, types have relations like *weakness* (does double damage), *resistance* (does halve damage) or *immunity* (does no damage) to manage effectiveness of attacks.
We do not want to model all of these relations, but begin with the *weakness* relation.

Additionally, we want to use the information about weaknesses between types to evaluate whether one Pokémon is at a disadvantage against another one. After creating the ontological backbone, we can express this using an SWRL rule.

In [ ]:
pkmn_type = pkmn_onto.PokemonType
pkmn = pkmn_onto.Pokemon

with pkmn_onto:
    class weakTo(ObjectProperty):
        domain = [pkmn_type]
        range = [pkmn_type]
        
    class disadvantageAgainst(ObjectProperty):
        domain = [pkmn]
        range = [pkmn]

# Getting the individuals for the types
Fire = pkmn_onto.Fire
Water = pkmn_onto.Water
Grass = pkmn_onto.Grass

# Adding the weakness relations between the three starter types
Fire.weakTo.append(Water)
Water.weakTo.append(Grass)
Grass.weakTo.append(Fire)
# Here one could add the relations for all other types...

In [ ]:
# Now we add the rules connecting the different relations:
with pkmn_onto:
    disadvance_rule = Imp()
    disadvance_rule.set_as_rule("""Pokemon(?p1), Pokemon(?p2), PokemonType(?t1), PokemonType(?t2), primaryType(?p1, ?t1), primaryType(?p2, ?t2), weakTo(?t1, ?t2) -> disadvantageAgainst(?p1, ?p2)""")
    # In natural language:
    # IF the primary type (t1) of one Pokemon (p1) is weak to the primary type (t2) of another Pokemon (p2) 
    # THEN Pokemon p1 is at a disadvantage against Pokemon p2

In [ ]:
# To actually apply the rule, we need to sync the reasoner (similar to Protégé)
with pkmn_onto:
    sync_reasoner_pellet(infer_property_values = True, infer_data_property_values = True)

# To verify that the rule works correctly, let's have a look at the Pokemon 'Chikorita', a grass type. Thus, it should be at a disadvantage against every Fire Type in the graph.
chikorita = pkmn_onto.Chikorita
chikorita.disadvantageAgainst

In [ ]:
# In a last step, we save the ontology to check with e.g. Protege
pkmn_onto.save(file = "PokemonOntologyDisadvantages.owl", format = "rdfxml")

## Exercise 01 - Pokémon Generations

For the first exercise, we want to diversify the simple *Pokémon* class using complex classes that use class axioms to automatically assing Pokémon to the generation they belong to ($\in [1, 4]$).
This classification is based on the national dex number of each Pokémon.
Below you can find the concrete numbers, please create four new classes that follow the general class axioms discussed in the last lecture.

Generation 1: $\in [001, 151]$ 

Generation 2: $\in [152, 251]$ 

Generation 3: $\in [252, 386]$

Generation 4: $\in [387, 493]$ 

<details>

<summary>Unsure how to define a complex class? Click here</summary>
    
You can create complex classes using the same structure as simple classes `class ClassName(Superclass):`. In the body of this class, you can then use the *equivalent_to* keyword, to describe the specifications that need to be fulfilled by the individual to belong to the complex class.
An example:
```python
class RoundApple(onto.Apple):
    equivalent_to = [onto.Apple & (onto.hasShape == onto.Round)]
```
</details>

In [ ]:
with pkmn_onto:    
    ### Add your code here



    # Run the reasoner
    sync_reasoner_pellet(infer_property_values = True, infer_data_property_values = True)

In [ ]:
# Save the ontology to check 
pkmn_onto.save(file = "PokemonOntologyEx01.owl", format = "rdfxml")

## Exercise 02 - Pokémon Stats

For the second exercise, we will be extending the ontology to cover the different stats of each Pokemon. There are six in total: HP, Attack, Defense, Special Attack, Special Defense and Speed. With these stats created, we can add a rule that automatically calculates the BST (Base Stat Total), which is the sum of all six stats. Based on this BST, we can decide whether a Pokemon is powerful (BST >= 550) or not (BST < 550). Please adapt the ontology accordingly and add the SWRL rules.

To verify your result, you can arbitrary stats to some Pokemon. If you want to be realistic, here are some Pokemon and their stats (in the order described above):

Abra = [25, 20, 15, 105, 55, 90]

Tyranitar = [100, 134, 110, 95, 100, 61]

Haunter = [45, 50, 45, 115, 55, 95]

In [ ]:
with pkmn_onto:    
    ### Add your code here



    # Run the reasoner
    sync_reasoner_pellet(infer_property_values = True, infer_data_property_values = True)

In [ ]:
# Save the ontology to check 
pkmn_onto.save(file = "PokemonOntologyEx02.owl", format = "rdfxml")